In [1]:
from utils.utils import *
from utils.utils_nn import *
from utils.utils_baseline_svm import *
from utils.Rnn_model import Rnn_model
# import kwargs dicts
from utils.kwargs_file import kwargs_neural_data_init, \
    kwargs_lin_data_init, kwargs_svm  # , kwargs_rnn

import numpy as np
np.random.seed(seed())

import tensorflow as tf
tf.reset_default_graph()

import os

from tensorflow.contrib import rnn

from sklearn.feature_extraction import DictVectorizer
from sklearn import svm

from collections import Counter, OrderedDict

import pandas as pd

import random
random.seed(seed())

import argparse

In [2]:
"""
Enable passing some keyword arguments from command line.
This does not affect the Jupyter notebook.
"""
# try:
parser = argparse.ArgumentParser()

parser.add_argument('--use_suggestions', action='store', dest='use_suggestions',
                    help='Should the algorithm use label suggestions '+\
                        '(False would mean using only the given labeled data)', 
                    type=str, 
                    default=None)

_StoreAction(option_strings=['--use_suggestions'], dest='use_suggestions', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='Should the algorithm use label suggestions (False would mean using only the given labeled data)', metavar=None)

In [3]:
# results = parser.parse_args()
args, unknown = parser.parse_known_args()

if args.use_suggestions is not None:
    kwargs_neural_data_init.use_suggestions = str(args.use_suggestions)

In [4]:
"""

####### IF UNSURE, SKIP THIS CELL #######


Plug in manually the suggestion similarity threshold.
This is a very hacky solution, for getting all of the results for different thresholds.
Done this way because sending a for loop to the cluster takes too much overhead time,
on the otherhand, a for loop will use up too much RAM on local machine.
"""
# 0.7
# 0.8
# 0.9
# F

### DONE:
# 0.8
# F

kwargs_neural_data_init.use_suggestions = 'F'
# make sure SVM follows
kwargs_lin_data_init.use_suggestions = kwargs_neural_data_init.use_suggestions

In [5]:
# import the correct kwargs file, based on suggestion args
suggestion_convert_dict = \
    {'0.7': '07', 
     '0.8': '08', 
     '0.9': '09', 
     'F': '10'}

suggestion_str = \
    suggestion_convert_dict[kwargs_neural_data_init.use_suggestions]

import_kwargs_str = \
    'from utils.kwargs_file_{} import kwargs_rnn_GRU, kwargs_rnn_LSTM'.format(suggestion_str)

exec(import_kwargs_str)  # execute import statement

In [6]:
x_feed_train, y_feed_train, x_feed_val, y_feed_val,\
    char_int, char_int_inv, label_int, label_int_inv, \
    statistics_dict =\
    data_load_preprocess(**kwargs_neural_data_init)

Not using label suggestion data
The are 2028 observations
Sampling from allowed 47 labels
47 labels in the validation set, with
783 potential observation to draw from.
177 observations sampled for validation
606 observations for training
The ratio of validation to *training* is about 0.292


In [7]:
#### check if there are any strange splits between training and validation
#### e.g. labels (training and validation) don't match exactly
#### or not enough (or too many) observation were set aside for validation
y_train_counter = Counter(y_feed_train)
y_train_labs = list(y_train_counter.keys())
y_train_labs.sort()

y_val_counter = Counter(y_feed_val)
y_val_labs = list(y_val_counter.keys())
y_val_labs.sort()

assert y_train_labs==y_train_labs, 'Labels in training and validation set do not match!'

validation_training_ratios = []
for label in y_train_labs:
    val_num = y_val_counter[label]
    train_num = y_train_counter[label]
    ratio = float(val_num / train_num)
    validation_training_ratios.append(ratio)
#     print('{:.3f}'.format(ratio), label, val_num, train_num)

print('Minimal validation to trainig ration is {:.3f}'.format(min(validation_training_ratios)))
print('Maximal validation to trainig ration is {:.3f}'.format(max(validation_training_ratios)))

Minimal validation to trainig ration is 0.222
Maximal validation to trainig ration is 0.333


In [8]:
# returns np.arrays to feed into tf model
# training data
X_train, _, Y_train = index_transorm_xy(x=x_feed_train, 
                                        y=y_feed_train, 
                                        char_int=char_int, 
                                        label_int=label_int, 
                                        n_class=statistics_dict['n_class'])

# validation data
X_val, _, Y_val = index_transorm_xy(x=x_feed_val, 
                                    y=y_feed_val, 
                                    char_int=char_int, 
                                    label_int=label_int, 
                                    n_class=statistics_dict['n_class'])

# write a metadata file for embeddings visualizer and create path string
embed_vis_path = write_embeddings_metadata(log_dir=kwargs_rnn_GRU.log_dir, 
                                           dictionary=char_int, 
                                           file_name='metadata.tsv')

In [9]:
kwargs_feed_dict_train = {'x': X_train, 'y': Y_train}
kwargs_feed_dict_test = {'x': X_val, 'y': Y_val}

kwargs_rnn_GRU = nice_dict({**kwargs_rnn_GRU, **statistics_dict})
kwargs_rnn_LSTM = nice_dict({**kwargs_rnn_LSTM, **statistics_dict})

In [ ]:
"""
Train GRU and LSTM models
"""

In [ ]:
##### GRU model
hparam_str = make_hparam_string(**kwargs_rnn_GRU)

rnn_GRU_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_GRU
                      )

rnn_GRU_model.train()
rnn_GRU_model.close_session()

In [ ]:
##### LSTM model
hparam_str = make_hparam_string(**kwargs_rnn_LSTM)

rnn_LSTM_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_LSTM
                      )

rnn_LSTM_model.train()
rnn_LSTM_model.close_session()

In [ ]:
"""
Collect MRR per ATC code
"""

In [10]:
collect_mrr_per_atc = []
collect_obs_num_per_atc = []

collected_results_per_atc_code_dir = 'rnn_results/results_per_atc_code/'

collected_obs_num_fname = collected_results_per_atc_code_dir + \
    'rnn_svm_obs_num_per_atc_code_sim_{}.p'.format(suggestion_str)
collected_mrr_fname = collected_results_per_atc_code_dir + \
    'rnn_svm_results_per_atc_code_sim_{}.p'.format(suggestion_str)

In [11]:
"""
Initialize SVM
"""

'\nInitialize SVM\n'

In [12]:
v = DictVectorizer(sparse=True)

# load data
x_train_svm, x_val_svm, \
    y_train_svm, y_val_svm, \
    allowed_ngrams = \
    data_load_preprocess(**{**kwargs_lin_data_init, 
                            **{'linear_counters': False}})

# create and fit classifier
# create a sparse X matrix with character and n-grams features
X_train_svm = v.fit_transform([Counter(elem) for elem in x_train_svm])
X_val_svm = v.transform([Counter(elem) for elem in x_val_svm])

print('X_train_svm (sparse) matrix, of size {} by {} has been created.'
      .format(X_train_svm.get_shape()[0], 
              X_train_svm.get_shape()[1]))  # vectorized

svm_clf = svm.SVC(**kwargs_svm)

# print(svm_clf)

svm_clf.fit(X_train_svm, y_train_svm)

Not using label suggestion data
The are 2028 observations
Sampling from allowed 47 labels
47 labels in the validation set, with
783 potential observation to draw from.
177 observations sampled for validation
606 observations for training
The ratio of validation to *training* is about 0.292
X_train_svm (sparse) matrix, of size 606 by 4373 has been created.


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=2178, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
# array to hold log probabilities (takes a bit longer to calc)
pred_prob_svm = svm_clf.predict_log_proba(X_val_svm)
# makes y into array with the same shape as the log prob
y_val_svm_dense = y_to_dense(y=y_val_svm, 
                         classes=svm_clf.classes_)

recip_rank_array_svm = \
    mean_reciprocal_rank(
        y_val_svm_dense, 
        pred_prob_svm, 
        average=False)

In [14]:
row_mrr = OrderedDict()
row_obs_num = OrderedDict()

# init a dict of empty lists
for atc in svm_clf.classes_:
    row_mrr[atc] = []
#     row_obs_num[atc] = []

# create lists of reciprocal ranks
for atc, recip_rank in zip(y_val_svm, recip_rank_array_svm):
    row_mrr[atc].append(recip_rank)

# calculate MRR
for k,v in row_mrr.items():
    row_mrr[k] = np.mean(v)
    row_obs_num[k] = len(v)

row_mrr['Similarity'] = int(suggestion_str) / 10
row_obs_num['Similarity'] = int(suggestion_str) / 10
row_mrr['Model type'] = 'SVM'
row_obs_num['Model type'] = 'SVM'

row_mrr.move_to_end('Model type', last=False)
row_obs_num.move_to_end('Model type', last=False)
row_mrr.move_to_end('Similarity', last=False)
row_obs_num.move_to_end('Similarity', last=False)

In [15]:
# collect results
collect_mrr_per_atc.append(row_mrr)
collect_obs_num_per_atc.append(row_obs_num)

In [ ]:
"""
Get MRR per ATC code
"""

In [ ]:
np.shape(kwargs_feed_dict_train['x'])

In [ ]:
kwargs_rnn_GRU.keys()

In [ ]:
kwargs_rnn_GRU.n_class

In [16]:
##### GRU model
hparam_str = make_hparam_string(**kwargs_rnn_GRU)

rnn_GRU_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_GRU
                      )

In [17]:
rnn_GRU_model.restore(kwargs_rnn_GRU.log_dir+hparam_str)

Loading variables from rnn_final_batch_trained_models/sim_10/GRU,bidir=F,noisy_tanh,learn_p=F,noise_alpha=0.9,noise_half_normal=T,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=1.0,one_hot,hidden_state_size=128,l2_weight_reg=1.0E-02,target_rep_weight=0.3/
Loading successful


In [18]:
##### LSTM model
hparam_str = make_hparam_string(**kwargs_rnn_LSTM)

rnn_LSTM_model = Rnn_model(hparam_str=hparam_str, 
                      embed_vis_path=embed_vis_path, 
                      feed_dict_train=kwargs_feed_dict_train, 
                      feed_dict_test=kwargs_feed_dict_test, 
                **kwargs_rnn_LSTM
                      )

In [19]:
rnn_LSTM_model.restore(kwargs_rnn_LSTM.log_dir+hparam_str)

Loading variables from rnn_final_batch_trained_models/sim_10/LSTM,bidir=F,noisy_tanh,learn_p=T,noise_alpha=1.15,noise_half_normal=T,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_weight_reg=1.0E-03,target_rep_weight=0.5/
Loading successful


In [20]:
def get_mrr_per_atc(Rnn_model_inst):
    """
    Returns two OrderedDict's, MRR per ATC code and number of observations per ATC code
    Assumes that th efile was run with the same suggestion settings and x,y inputs
    """
    
    [accuracy, cost, recip_rank, top_k, logits] = \
        Rnn_model_inst.run_eval()

    # get an array of reciprocal ranks
    with tf.Session() as sess:
        recip_rank_array = \
            Rnn_model_inst.get_reciprocal_rank(
                logits=logits, 
                targets=kwargs_feed_dict_test['y']).eval()

    row_mrr = OrderedDict()
    
    row_obs_num = OrderedDict()
    row_obs_num['Similarity'] = int(suggestion_str) / 10
    row_obs_num['Model type'] = Rnn_model_inst.rnn_type
    
    # init a dict of empty lists
    for atc in set(y_feed_val):
        row_mrr[atc] = []

    # create lists of reciprocal ranks
    for atc, recip_rank in zip(y_feed_val, recip_rank_array):
        row_mrr[atc].append(recip_rank)

    # calculate MRR
    for k,v in row_mrr.items():
        row_mrr[k] = np.mean(v)
        row_obs_num[k] = len(v)

    row_mrr['Similarity'] = int(suggestion_str) / 10
    row_mrr['Model type'] = Rnn_model_inst.rnn_type

    row_mrr.move_to_end('Model type', last=False)
    row_mrr.move_to_end('Similarity', last=False)
    
    return row_mrr, row_obs_num

In [21]:
row_mrr_GRU, row_obs_num_GRU = \
    get_mrr_per_atc(rnn_GRU_model)

collect_mrr_per_atc.append(row_mrr_GRU)
collect_obs_num_per_atc.append(row_obs_num_GRU)

In [22]:
row_mrr_LSTM, row_obs_num_LSTM = \
    get_mrr_per_atc(rnn_LSTM_model)

collect_mrr_per_atc.append(row_mrr_LSTM)
collect_obs_num_per_atc.append(row_obs_num_LSTM)

In [23]:
# save the 3 results (SVM, LSTM, GRU) to pickle file format
# for each of the two statistics (MRR and count)
save(fname=collected_mrr_fname, 
     obj=collect_mrr_per_atc)

save(fname=collected_obs_num_fname, 
     obj=collect_obs_num_per_atc)